# Naive Bayes Classification

The dataset was obtained from https://www.kaggle.com/nltkdata/movie-review.

In [1]:
import numpy as np
import pandas as pd
import re
from collections import Counter

In [2]:
# read data and parse it
data = pd.read_csv('../datasets/sms_spam.csv', names=['tag', 'text'])
data.text = [str(doc) for doc in data['text']]
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5574 entries, 0 to 5573
Data columns (total 2 columns):
tag     5574 non-null object
text    5574 non-null object
dtypes: object(2)
memory usage: 87.2+ KB


In [3]:
# Splitting data 70 % train, 30 % test
train_df = data.iloc[:3000, :]
test_df = data.iloc[3001:, :]

In [10]:
class NaiveBayesText:
    
    def __init__(self, train_df):
        """
        train_df (pandas.DataFrame) : Training dataset
        train_df must have a column called 'tag' representing the output class,
            and a column called 'text'
        """
        self.train_df = train_df
        # pre-processing to remove punctuations
        self.train_df['text'] = [
            re.sub('\d|\"|\'|!|\)|\(|\.|,|\\b', '', doc).split() 
            for doc in train_df.text.values]
        self.tags = list(set(train_df.tag.values))
        self.total_vocabulary_size = len(self.get_vocabulary(self.train_df))
        self.tag_vocabulary_counts = {tag : Counter(self.get_vocabulary(
            self.train_df.loc[self.train_df['tag'] == tag])) for tag in self.tags}
    
    def get_vocabulary(self, df):
        vocabulary = []
        for doc_list in df.text.values:
            for word in doc_list:
                vocabulary.append(word)
        return vocabulary
    
    def prob_tag(self, tag):
        return len(self.train_df.loc[self.train_df['tag'] == tag])/len(self.train_df)

    def prob_doc_given_tag(self, doc, tag, alpha):
        prob_words = np.array([self.prob_word_given_tag(word, tag, alpha) 
                               for word in doc.split()])
        return np.prod(prob_words) * self.prob_tag(tag)

    def prob_word_given_tag(self, word, tag, alpha):
        word_count = self.tag_vocabulary_counts[tag][word]
        return (word_count + alpha)/(len(self.tag_vocabulary_counts) + alpha * 
                                          self.total_vocabulary_size)

    def predict_tag(self, doc, alpha):
        prob_tags = np.array([self.prob_doc_given_tag(doc, tag, alpha) for tag in self.tags])
        return self.tags[np.argmax(prob_tags)]
    
    def predict_batch(self, docs, print_progress = False, alpha=1):
        predict_tags = []
        i = 0
        for doc in docs:
            predict_tags.append(self.predict_tag(doc, alpha))
            if print_progress and i % len(docs)//10 == 0:
                print("Predicted {} of {} documents".format(i * len(docs)//10, len(docs)))
            i += 1
        if print_progress:
            print("Predicted all documents\n")
        return predict_tags

In [11]:
def get_best_alpha(train_data):
    alpha_values = [x/100 for x in range(500, 600, 5)]
    accuracies = np.zeros([len(alpha_values), len(train_data.fold_id.values)])
    for fold in range(len(train_data.fold_id.values)):
        test = train_data.loc[train_data['fold_id'] == fold]
        train = train_data.loc[train_data['fold_id'] != fold]
        nbt = NaiveBayesText(train)
        if len(test['tag']) == 0:
            continue
        i = 0
        for a in alpha_values:
            y_hat = nbt.predict_batch(test['text'], alpha=a)
            accuracy = calculate_accuracy(y_hat, test['tag'])
            accuracies[i][fold] = accuracy
            i += 1
    alpha_accuracies = accuracies.mean(axis=1)
    for j in range(len(alpha_values)):
        print("validation accuracy for alpha = {} is {}"
              .format(alpha_values[j], alpha_accuracies[j]))
    return alpha_values[np.argmax(alpha_accuracies)]

def calculate_accuracy(y_hat, y):
    return sum(1 for a, b in zip(y_hat,y) if a == b) / len(y)

In [12]:
# best_alpha = get_best_alpha(train_df)
# Use the best parameter
nbt = NaiveBayesText(train_df)
y_hat = nbt.predict_batch(test_df['text'], print_progress=True)
test_acc = calculate_accuracy(test_df['tag'], y_hat)
print("Test set accuracy: {:1.3f}%".format(test_acc * 100))

/home/pranaymethuku/.local/lib/python3.5/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


Predicted 0 of 2573 documents
Predicted 257 of 2573 documents
Predicted 514 of 2573 documents
Predicted 771 of 2573 documents
Predicted 1029 of 2573 documents
Predicted 1286 of 2573 documents
Predicted 1543 of 2573 documents
Predicted 1801 of 2573 documents
Predicted 2058 of 2573 documents
Predicted 2315 of 2573 documents
Predicted all documents

Test set accuracy: 90.983%
